In [ ]:
import os
from openai import OpenAI, pydantic_function_tool
import rich
import requests
import json
from pydantic import BaseModel, Field
from google.colab import userdata

In [ ]:
api_key = userdata.get('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI(api_key=api_key)

**Calling the same function multiple times when response from Chat API and Responses API requires to call function multiple times**

The Pydantic-generated function structure is acceptable in OpenAI's Chat API, but the Responses API requires a slightly different structure.

In [ ]:
class GetWeather(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    print(f"get_weather function called to get weather for latitude = {latitude}, longitude = {longitude}")
    print(f"And result is  = {data['current']['temperature_2m']}")
    return data['current']['temperature_2m']

# Notice the function property in the output, which is not acceptable in Responses API
rich.print(pydantic_function_tool(GetWeather))

{
    'type': 'function',
    'function': {
        'name': 'GetWeather',
        'strict': True,
        'parameters': {
            'properties': {
                'latitude': {'description': 'Latitude of the location', 'title': 'Latitude', 'type': 'number'},
                'longitude': {'description': 'Longitude of the location', 'title': 'Longitude', 'type': 'number'}
            },
            'required': ['latitude', 'longitude'],
            'title': 'GetWeather',
            'type': 'object',
            'additionalProperties': False
        }
    }
}

# Chat Completion API

https://platform.openai.com/docs/guides/function-calling?api-mode=chat

First Step where model will responed with tool call request

In [ ]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city. Response should be courteous and professional."},
    # {"role": "user", "content": "What's the weather like in Karachi and Lahore?"}
    # {"role": "user", "content": "NYC"}
    {"role": "user", "content": "Berlin and Paris"} # This time we are sending multiple cities
]
tools = [pydantic_function_tool(GetWeather)]
response = openai.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools = tools
)

rich.print(response.choices[0])
print("Finish Reason = ", response.choices[0].finish_reason)
# Chat API returns tool_calls for all expected tools in single response
rich.print(response.choices[0].message.tool_calls)
print("Number of tool calls: ",len(response.choices[0].message.tool_calls))

Choice(
    finish_reason='tool_calls',
    index=0,
    logprobs=None,
    message=ChatCompletionMessage(
        content=None,
        refusal=None,
        role='assistant',
        annotations=[],
        audio=None,
        function_call=None,
        tool_calls=[
            ChatCompletionMessageToolCall(
                id='call_bPBYEADmIi7Q3ETIF5YGQT4g',
                function=Function(arguments='{"latitude": 52.52, "longitude": 13.405}', name='GetWeather'),
                type='function'
            ),
            ChatCompletionMessageToolCall(
                id='call_m7KVrHRU7nGz99mM1Xiv5UFP',
                function=Function(arguments='{"latitude": 48.8566, "longitude": 2.3522}', name='GetWeather'),
                type='function'
            )
        ]
    )
)

Finish Reason =  tool_calls


[
    ChatCompletionMessageToolCall(
        id='call_bPBYEADmIi7Q3ETIF5YGQT4g',
        function=Function(arguments='{"latitude": 52.52, "longitude": 13.405}', name='GetWeather'),
        type='function'
    ),
    ChatCompletionMessageToolCall(
        id='call_m7KVrHRU7nGz99mM1Xiv5UFP',
        function=Function(arguments='{"latitude": 48.8566, "longitude": 2.3522}', name='GetWeather'),
        type='function'
    )
]

Number of tool calls:  2


Second Step where we are calling the `get_weather` function and sending the response back to Chat API.

But this time we are running a loop on `tool_call` to call all the functions

In [ ]:
# Defining a function in which we will loop through the tool_calls,
# invoke the corresponding functions, and update the response.
# Plus, we will append a new message for each function call to promptMessages.
def handle_tool_call(promptMessages, responseMessage):
    # Looping on tool_calls
    for tool_call in responseMessage.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        weather = get_weather(**arguments)
        new_message = {
            "role": "tool",
            "content": str(weather),
            "tool_call_id": tool_call.id
        }
        promptMessages.append(new_message)

In [ ]:
if response.choices[0].finish_reason == "tool_calls": # Check if finish_reason is tool_calls
    # Important: first we will append the previous message (response.choices[0].message) in
    # message history and then iterate through tool_calls
    messages.append(response.choices[0].message)
    handle_tool_call(messages, response.choices[0].message)
    response2 = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print("Model Response2 = ",response2.choices[0].message.content)
    print("Finish Reason = ",response2.choices[0].finish_reason)

get_weather function called to get weather for latitude = 52.52, longitude = 13.405
And result is  = 7.8
get_weather function called to get weather for latitude = 48.8566, longitude = 2.3522
And result is  = 13.5
Model Response2 =  The current weather updates are as follows:

- **Berlin:** The temperature is approximately 7.8°C.
- **Paris:** The temperature is around 13.5°C.

If you need any further information, feel free to ask!
Finish Reason =  stop


# Responses API

https://platform.openai.com/docs/guides/function-calling?api-mode=responses

In [ ]:
rich.print(pydantic_function_tool(GetWeather))

{
    'type': 'function',
    'function': {
        'name': 'GetWeather',
        'strict': True,
        'parameters': {
            'properties': {
                'latitude': {'description': 'Latitude of the location', 'title': 'Latitude', 'type': 'number'},
                'longitude': {'description': 'Longitude of the location', 'title': 'Longitude', 'type': 'number'}
            },
            'required': ['latitude', 'longitude'],
            'title': 'GetWeather',
            'type': 'object',
            'additionalProperties': False
        }
    }
}

To use pydantic-generated function, we need to use `openai.responses.parse()` function call

https://github.com/openai/openai-python/blob/main/examples/responses/structured_outputs_tools.py

Sometimes Responses API is not sending weather function call for all cities in one go.
It is totally depends on model how it will organized the call of function, all in one go or one by one

In [ ]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant and provide update on weather in a city. Response should be courteous and professional."},
    # {"role": "user", "content": "What's the weather like in Karachi and Lahore?"}
    {"role": "user", "content": "Berlin and Paris"}
]
# messages=[
#     {"role": "developer", "content": "You are a helpful assistant and provide update on weather in all the cities as asked by user. Response should be courteous and professional."},
#     {"role": "user", "content": "What's the weather in following cities Karachi and Lahore?"}
# ]
tools = [pydantic_function_tool(GetWeather)]

# Sometimes Responses API is not sending weather function call for all cities in one go.
# It is totally depends on model how it will organized the call of function, all in one go or one by one
response = openai.responses.parse(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output)
# In this case Responses API calling tools one by one. We need to send the response for the first
# tool call to the API, after which the Responses API will send the next tool call.

print("Number of tool calls: ",len(response.output))
# rich.print(response)

Status =  completed



[
    ParsedResponseFunctionToolCall(
        arguments='{"latitude":52.52,"longitude":13.405}',
        call_id='call_SpYMdNt34SA54MaRSk7mcwlT',
        name='GetWeather',
        type='function_call',
        id='fc_67e2eda6f49081928d5110677d234efd04e239c703fbd525',
        status='completed',
        parsed_arguments=GetWeather(latitude=52.52, longitude=13.405)
    ),
    ParsedResponseFunctionToolCall(
        arguments='{"latitude":48.8566,"longitude":2.3522}',
        call_id='call_vDVZ82Eb6jz7Q5883QN99oHT',
        name='GetWeather',
        type='function_call',
        id='fc_67e2eda76fe08192a4111de8b190a0e704e239c703fbd525',
        status='completed',
        parsed_arguments=GetWeather(latitude=48.8566, longitude=2.3522)
    )
]

Number of tool calls:  2


In case of weather function it was random for me so we are sing example of different function to test multiple calls of same function in single response

In [ ]:
class SendEmail(BaseModel):
    to: str = Field(..., description="Email address of the recipient")
    subject: str = Field(..., description="Subject of the email")
    body: str = Field(..., description="Body of the email")


def send_email(to, subject, body):
    print(f"Tool send_email Sending email to {to} with subject {subject}")
    print(f"Body: {body}")
    print(f"Email Tool call completed")
    return "Email Sent"

### Using old way of sending history messages in every call

First Step where model will responed with tool call request

In [ ]:
messages=[
    {"role": "developer", "content": "You are a helpful assistant, you can send email about weather in a city. Email should be courteous and professional."},
    {"role": "user", "content": "send an email to first@gmail.com and second@gmail.com saying Hello"}
]
tools = [pydantic_function_tool(SendEmail)]

response = openai.responses.parse(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output)

print("Number of tool calls: ",len(response.output))
# rich.print(response)

Status =  completed



[
    ParsedResponseFunctionToolCall(
        arguments='{"to":"first@gmail.com","subject":"Hello","body":"Hello"}',
        call_id='call_86ZoIPJcqk5pEJ0sHJwk31wZ',
        name='SendEmail',
        type='function_call',
        id='fc_67e2edd21cf4819289f270631f84be2b079aae0081256c0a',
        status='completed',
        parsed_arguments=SendEmail(to='first@gmail.com', subject='Hello', body='Hello')
    ),
    ParsedResponseFunctionToolCall(
        arguments='{"to":"second@gmail.com","subject":"Hello","body":"Hello"}',
        call_id='call_ER1bZVvIHJ4KUie1Oc9k6jWR',
        name='SendEmail',
        type='function_call',
        id='fc_67e2edd272048192aa7ce16b317c9b42079aae0081256c0a',
        status='completed',
        parsed_arguments=SendEmail(to='second@gmail.com', subject='Hello', body='Hello')
    )
]

Number of tool calls:  2


Second Step where we are calling the `send_email` function and sending the response back to Responses API.

But this time we are running a loop on `response.output` to call all the functions

In [ ]:
# Defining a function in which We will call the relevant function
# and return the new message object
def handle_tool_call_responses(tool_call):
    result = send_email(tool_call.parsed_arguments.to, tool_call.parsed_arguments.subject,tool_call.parsed_arguments.body)

    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    }
    return new_message

In [ ]:
# Logic here is bit different then what we have done in Chat API
tool_call_results = []
for tool_call in response.output:
    if tool_call.type == "function_call": # check if the output's type is function_call
        # Adding the result of tool calls individually
        tool_call_results.append(handle_tool_call_responses(tool_call))

# deleting the parsed_arguments from the response.output because to
# sending messages as history to the API, we need to remove the parsed_arguments
for item in response.output:
    del item.parsed_arguments

# For multiple tool calls, we need to append the output of each tool call individually.
# This is different from Chat API in which we can just send message object which have all the tool calls
# extend function it adds each item from the iterable one by one.
messages.extend(response.output)
messages.extend(tool_call_results) # Adding the result of tool calls individually
# rich.print(messages)
response2 = openai.responses.parse(model=MODEL, input=messages,tools = tools)
print("Model Response2 output_text = ",response2.output_text)
# rich.print("Model Response2 = ",response2)
print("Status = ",response2.status)

Tool send_email Sending email to first@gmail.com with subject Hello
Body: Hello
Email Tool call completed
Tool send_email Sending email to second@gmail.com with subject Hello
Body: Hello
Email Tool call completed
Model Response2 output_text =  The emails saying "Hello" have been successfully sent to both first@gmail.com and second@gmail.com. If you need anything else, feel free to ask!
Status =  completed


### Using new way of conversation state by sending perivous reponse id

First Step where model will responed with tool call request

In [ ]:
# This section is same as above
messages=[
    {"role": "developer", "content": "You are a helpful assistant, you can send email about weather in a city. Email should be courteous and professional."},
    {"role": "user", "content": "send an email to first@gmail.com and second@gmail.com saying Hello"}
]
tools = [pydantic_function_tool(SendEmail)]

response = openai.responses.parse(
    model=MODEL,
    input=messages,
    tools = tools
)

print("Status = ",response.status) # Status will not indicate the tool call
print(response.output_text) # Empty
rich.print(response.output)

print("Number of tool calls: ",len(response.output))
# rich.print(response)

Status =  completed



[
    ParsedResponseFunctionToolCall(
        arguments='{"to":"first@gmail.com","subject":"Hello","body":"Hello!"}',
        call_id='call_jgGqQFnEgg5A0oLCQoUZTHtM',
        name='SendEmail',
        type='function_call',
        id='fc_67e2ee2c2f948192bcc1033c65ce3d6e07a988a7a0586cab',
        status='completed',
        parsed_arguments=SendEmail(to='first@gmail.com', subject='Hello', body='Hello!')
    ),
    ParsedResponseFunctionToolCall(
        arguments='{"to":"second@gmail.com","subject":"Hello","body":"Hello!"}',
        call_id='call_bctngnqF2NSWIIZNA8uz9E76',
        name='SendEmail',
        type='function_call',
        id='fc_67e2ee2c73c08192b4d147c17f9946df07a988a7a0586cab',
        status='completed',
        parsed_arguments=SendEmail(to='second@gmail.com', subject='Hello', body='Hello!')
    )
]

Number of tool calls:  2


Second Step where we are calling the `send_email` function and sending the response back to Responses API.

But this time we are running a loop on `response.output` to call all the functions

In [ ]:
# This section is same as above

# Call the relevant function and return the output
def handle_tool_call_responses(tool_call):
    result = send_email(tool_call.parsed_arguments.to, tool_call.parsed_arguments.subject,tool_call.parsed_arguments.body)

    new_message = {
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    }
    return new_message

The only difference in below section is how messages are sent.

In [ ]:
# The only difference in this section is how messages are sent.

# Logic here is bit different then what we have done in Chat API
tool_call_results = []
for tool_call in response.output:
    if tool_call.type == "function_call": # check if the output's type is function_call
        tool_call_results.append(handle_tool_call_responses(tool_call))

# Not needed now because we are using response.id
# for item in response.output:
#     del item.parsed_arguments

# Not needed now because we are using response.id
# messages.extend(response.output)

# Emptying the messages array because we are sending the previous response id,
# therefore we don't need to send the previous message
messages = []
# Adding the result of tool calls individually
messages.extend(tool_call_results)
# rich.print(messages)
response2 = openai.responses.parse(model=MODEL, input=messages,tools = tools, previous_response_id=response.id)
print("Model Response2 output_text = ",response2.output_text)
# rich.print("Model Response2 = ",response2)
print("Status = ",response2.status)

Tool send_email Sending email to first@gmail.com with subject Hello
Body: Hello!
Email Tool call completed
Tool send_email Sending email to second@gmail.com with subject Hello
Body: Hello!
Email Tool call completed
Model Response2 output_text =  I have sent an email saying "Hello!" to both first@gmail.com and second@gmail.com. If you need further assistance, feel free to ask!
Status =  completed
